# 💳 Projeto de Detecção de Fraudes em Cartões de Crédito


Este projeto utiliza aprendizado de máquina supervisionado para identificar transações financeiras fraudulentas, com base em um conjunto de dados do Kaggle.
A abordagem segue a metodologia CRISP-DM e utiliza validação cruzada, SMOTE, TF-IDF e modelos como Regressão Logística e Floresta Aleatória.


## 📚 Importação das Bibliotecas

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

import pickle
import warnings
warnings.filterwarnings("ignore")


## 📥 Carregamento do Dataset

In [ ]:

url = "https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv"
df = pd.read_csv(url)
df.shape


## ✂️ Divisão do Conjunto de Dados (60/20/20)


- 60% para treino (com balanceamento SMOTE)
- 20% para validação
- 20% para teste
A divisão é estratificada para manter a proporção da classe alvo.


In [ ]:

X = df.drop("Class", axis=1)
y = df["Class"]

# 20% Teste, 80% Treino + Validação
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Dos 80%, separa 25% para validação → 20% total
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)


## ⚖️ Normalização com StandardScaler

In [ ]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


## 🧪 Aplicação do SMOTE no Conjunto de Treinamento

In [ ]:

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)


## 🤖 Modelos de Classificação

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, model in models.items():
    print(f"\n{name}")
    model.fit(X_train_bal, y_train_bal)
    y_pred = model.predict(X_val_scaled)
    print(classification_report(y_val, y_pred))
    ConfusionMatrixDisplay.from_estimator(model, X_val_scaled, y_val)
    plt.title(name)
    plt.show()


## 💾 Salvando o Melhor Modelo com Pickle

In [ ]:

best_model = models["Random Forest"]
with open("modelo_random_forest.pkl", "wb") as f:
    pickle.dump(best_model, f)
